Working version of Namsor Application 03-07-2024.

This program takes existing data from the xlsx files compiled for each Law Review and extracts the name data from each author. The names are then passed to the Namsor API to associate each name with a gender, along with a calculated probability of that classification.
The determined gender for each name, as well as the probability are then added back to the xlsx data file for each respective Law Review. - F

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
law_reviews_names = {
    'BuffLR': 'Buffalo Law Review', #DONE
    'CaliLR': 'California Law Review', #citation line issue
    'CWRLR': 'Case Western Reserve Law Review', #DONE
    'CathULR': 'Catholic University Law Review', #DONE
    'ChiKLR': 'Chicago-Kent Law Review', #DONE
    'ClevSLR': 'Cleveland State Law Review', #DONE
    'CorLR': 'Cornell Law Review', #DONE (readme)
    'DePLR': 'DePaul Law Review', #DONE
    'DiLR': 'Dickinson Law Review (Penn State)', #hein has files
    'FLRL': 'Florida Law Review', #DONE
    'FordLR': 'Fordham Law Review', #DONE
    'HastLJ': 'Hastings Law Journal', #DONE
    'IndLJ': 'Indiana Law Journal', #DONE
    'KentuLLJ': 'Kentucky Law Journal', #DONE
    'LouisLR': 'Louisiana Law Review', #DONE
    'MarqLR': 'Marquette Law Review', #DONE
    'MichLR': 'Michigan Law Review', #DONE
    'MinnLR': 'Minnesota Law Review', #DONE
    'MissLR': 'Missouri Law Review', #DONE
    'MontLR': 'Montana Law Review', #DONE
    'NCarolLR': 'North Carolina Law Review', ##DONE
    'NDakoLR': 'North Dakota Law Review', #DONE(special case notebook)
    'NotDamLR': 'Notre Dame Law Review',#DONE
    'SMULR': 'SMU Law Review',#DONE
    'SCarolLR': 'South Carolina Law Review', #DONE
    'SJohnLR': "St. John's Law Review",#DigiCmns do not have recommended citation
    'UChiLR': 'University of Chicago Law Review',#No citation page
    'UMiaLR': 'University of Miami Law Review', #DONE
    'VandLR': 'Vanderbilt Law Review', #DONE
    'WashLeeLR': 'Washington & Lee Law Review', #DONE
    'WashLR': 'Washington Law Review',#DONE
    'MaryLR': 'Maryland Law Review', #DONE
    'DenvLR': 'Denver Law Review' #DONE
}

# IMPORTS AND GLOBALS

In [6]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import os
#/content/drive/MyDrive/LawRevProject/Data/Washington Law Rev/XLSX/WashLR__21_05_2024_23_11.xlsx
NAMSOR_API_KEY = "API_KEY"
NAMSOR_URL = 'https://v2.namsor.com/NamSorAPIv2/api2/json/genderFullBatch'
unique_key = 'WashLR'
LR_General_Path = "/content/drive/MyDrive/LawRevProject/Data/Washington Law Rev"
xlsx_file = f'{LR_General_Path}/XLSX/WashLR__21_05_2024_23_11.xlsx'
log_path = f'{LR_General_Path}/Logs'
namsor_dir = f'{LR_General_Path}/Namsor'
max_num_authors = 0

invalid_names = {
    'BuffLR': ['buffalo law review', 'buffalo law review board'],
    'CWRLR': ['The Editor', 'The Editors','WESTERN RESERVE LAW REVIEW'],
    'ChiKLR': ['CHICAGO-KENT LAW REVIEW'],
    'ClevSLR': ['Index title page', 'INDEX BY SUBJECTS', 'INDEX BY AUTHORS', 'NOTE', 'STANDARDS OF TITLE EXAMINATION', 'CLEVELAND-MARSHALL LAW SCHOOL',
                'TABLE OF CONTENTS - ISSUE 1', '1(', 'CLEVELAND STATE LAW REVIEW', 'COMMENT', 'CASE COMMENT'],
    'CorLR': ['JOURNAL STAFF', 'NOTES AND COMMENT', 'ALUMNI NOTES', 'LAW SCHOOL NOTES', 'EDITORIAL', 'BOOK REVIEWS'],
    'HastLJ': ['UC HASTINGS COLLEGE OF THE LAW', 'HASTINGS LAW JOURNAL'],
    'MontLR':['RECENT DECISIONS AFFECTING THE MONTANA PRACTITIONER', 'LEGAL SHORTS'],
    'SCarolLR': ['TRANSCRIPT FORM PROFESSIONALISM CONFERENCE', 'ADDENDUM', 'APPENDIX: PROFESSIONALISM INITIATIVES', 'AGENDA', 'TRANSCRIPT FROM PROFESSIONALISM CONFERENCE',
                 'GRANDPARTEN VISITATION RIGHTS IN SOUTH CAROLINA IN THE WAKE OF TROXEL V. GRANVILLE', 'CONGRESSIONAL CONTROL OF FEDERAL COURT JURISDICTION: THE CASE STUDY OF ABORTION','INTRODUCTION', 'LEADING ARTICLES', 'COMMENTS', 'SURVEY OF S.C. LAW'
                 'PREFACE'],
    'NotDamLR': ['NOTRE DAME'],
    'VandLR': ['LESSONS FROM STUDYING THE INTERNATIONAL ECONOMICS OF INTELLECTUAL PROPERTY RIGHTS', ''],
    'MissLR': ['TITLE PAGE', 'TABLE OF CONTENTS - ISSUE 1', 'TABLE OF CONTENTS - ISSUE 2', 'TABLE OF CONTENTS - ISSUE 3', 'TABLE OF CONTENTS - ISSUE 4', 'INDEX', 'BOOK REVIEWS', 'RECENT CASES', 'INDEX TO SUBJECTS'],
    'DePLR': [],
    'NDakoLR': ['North Dakota Law Review Associate Editors', 'North Dakota Law Review', 'American Bar Association Journal'],
    'IndLJ': [],
    'MarqLR': ['MARQUETTE UNIVERSITY'],
    'KentuLLJ': ['Kentucky'],
    'FordLR': ['EDITORS', 'NOTES', 'BOOK', 'NEW YORK', 'BUSINESS', 'REVIEW', ' LAW ', 'LIABILITY', 'LEGISLATION', 'RECENT', 'COMMITTEE', 'TRIAL', 'BIBLIOGRAPHY'],
    'MinnLR' : ['Minn. L. Rev.'],
    'LouisLR': ['TABLE OF CONTENTS', 'University', 'Louisiana', 'Appellate', 'Government', 'Contents', 'Masthead'],
    'MaryLR': ['Maryland', 'Title', 'Page', 'Book', 'Reviews', 'Legislation', 'Editorial', 'Table', 'Contents', 'Notes', ' Law ', 'Committee', 'Trial', 'Index', ' Laws ', 'Defendant', 'Liablity', 'Criminal', 'Government', 'Federal', 'Evidence', 'Tort', 'Contract', ' Court ', 'Constitutional', 'Constitution'],
    'DenvLR': ['Denver', 'Colorado', 'Title', 'Page', 'Book', 'Reviews', 'Legislation', 'Editorial', 'Table', 'Contents', 'Notes', ' Law ', 'Committee', 'Trial', 'Index', ' Laws ', 'Defendant', 'Liablity', 'Criminal', 'Government', 'Federal', 'Evidence', 'Tort', 'Contract', ' Court ', 'Constitutional', 'Constitution', 'Legal', 'Dicta', 'Dictaphun'],
    'CathULR': ['Catholic', 'University', ' Law ', 'Review', 'Title', 'Page', 'Book', 'Reviews', 'Legislation', 'Editorial', 'Table', 'Contents', 'Notes', ' Law ', 'Committee', 'Trial', 'Index', ' Laws ', 'Defendant', 'Liablity', 'Criminal', 'Government', 'Federal', 'Evidence', 'Tort', 'Contract', ' Court ', 'Constitutional', 'Constitution'],
    'FLRL': ['University', ' Law ', 'Review', 'Title', 'Page', 'Book', 'Reviews', 'Legislation', 'Editorial', 'Table', 'Contents', 'Notes', ' Law ', 'Committee', 'Trial', 'Index', ' Laws ', 'Defendant', 'Liablity', 'Criminal', 'Government', 'Federal', 'Evidence', 'Tort', 'Contract', ' Court ', 'Constitutional', 'Constitution'],
    'SMULR': ['University', ' Law ', 'Review', 'Title', 'Page', 'Book', 'Reviews', 'Legislation', 'Editorial', 'Table', 'Contents', 'Notes', ' Law ', 'Committee', 'Trial', 'Index', ' Laws ', 'Defendant', 'Liablity', 'Criminal', 'Government', 'Federal', 'Evidence', 'Tort', 'Contract', ' Court ', 'Constitutional', 'Constitution'],
    'NCarolLR': ['North Carolina','University', ' Law ', 'Review', 'Title', 'Page', 'Book', 'Reviews', 'Legislation', 'Editorial', 'Table', 'Contents', 'Notes', ' Law ', 'Committee', 'Trial', 'Index', ' Laws ', 'Defendant', 'Liablity', 'Criminal', 'Government', 'Federal', 'Evidence', 'Tort', 'Contract', ' Court ', 'Constitutional', 'Constitution'],
    'UMiaLR': ['Miami','Florida','Attorney', 'Summary', 'Citations','Torts','University', ' Law ', 'Review', 'Title', 'Page', 'Book', 'Reviews', 'Legislation', 'Editorial', 'Table', 'Contents', 'Notes', ' Law ', 'Committee', 'Trial', 'Index', ' Laws ', 'Defendant', 'Liablity', 'Criminal', 'Government', 'Federal', 'Evidence', 'Tort', 'Contract', ' Court ', 'Constitutional', 'Constitution', 'Masthead'],
    'MichLR': ['Michigan','Attorney', 'Summary', 'Citations','Torts','University', ' Law ', 'Review', 'Title', 'Page', 'Book', 'Reviews', 'Legislation', 'Editorial', 'Table', 'Contents', 'Notes', ' Law ', 'Committee', 'Trial', 'Index', ' Laws ', 'Defendant', 'Liablity', 'Criminal', 'Government', 'Federal', 'Evidence', 'Tort', 'Contract', ' Court ', 'Constitutional', 'Constitution', 'Masthead', 'Statute', 'Illegal', 'Liability'],
    'WashLR': ['Washington Law', 'Anon','Attorney', 'Summary', 'Citations','Torts','University', ' Law ', 'Review', 'Title', 'Page', 'Book', 'Reviews', 'Legislation', 'Editorial', 'Table', 'Contents', 'Notes', ' Law ', 'Committee', 'Trial', 'Index', ' Laws ', 'Defendant', 'Liablity', 'Criminal', 'Government', 'Federal', 'Evidence', 'Tort', 'Contract', ' Court ', 'Constitutional', 'Constitution', 'Masthead', 'Statute', 'Illegal', 'Liability'],



}

# **Functions**

In [7]:

def get_author_names_from_data(df):
    '''
    :param df: dataframe containing the original data from law review xlsx file
    :return: dictionary containing just the author names extracted from the file to give to the Namsor API
    '''
    names = []
    namesDict = {}
    skip_names_list = invalid_names[unique_key]
    index = 0
    invalid_flag = False

    for row in df.itertuples():
        name = row.authors

        #check if name is NaN or None first
        if pd.isna(name):
          index += 1
          continue

        name = str(name).strip()


        for skip_name in skip_names_list:
          if skip_name.lower() in name.lower():
            print(f'skipping name: {name}')
            index += 1
            invalid_flag = True
            break

        if invalid_flag:
          invalid_flag = False
          continue
        #some entries are incorrect, so we skip them
        # if name in skip_names_list or name.lower() == 'nan' or name == None:
        #     index += 1
        #     continue

        #split multiple authors and enter each author into dictionary
        #split on ';' first... sometimes the names are separated by 'AND', so check if it is separated by 'AND'
        if ';' in name:
          current_authors_list = name.split(';')
        else:
          current_authors_list = name.split(' AND ')


        find_max_num_authors(current_authors_list)

        for current_author in current_authors_list:
          namesDict[current_author.strip()] = None
          names.append(current_author)



        index += 1


    return namesDict


def namsor_request(namesDict):
    '''

    :param namesDict(dictionary): dictionary of author names that we will pass to the Namsor API
    :return: respinse object from the Namsor API (json)
    '''


    headers = {
        "X-API-KEY": NAMSOR_API_KEY,
        "Accept": "application/json",
        "Content-Type": "application/json"

    }

    '''
    payload is a dictionary with a key of personalNames and a
    value of a list of dictionaries, each dictionary containing the data for each name
    for the Genderize Full Name tool we can pass a full name
    see docs for reference https://namsor.app/api-documentation/#genderize-full-name-batch
    '''


    #loop through the authorNames list and for each name create a new dictionary
    #add each dictionary to the personalNameList

    personal_name_list = [] #list of dictionaries containing the items for payload


    #loop through names in the namesDict and create a new dictionary for the payload
    for name in namesDict:
      author_dict = {'name': name}
      personal_name_list.append(author_dict)



    #create payload using the list of dictionaries
    payload = {'personalNames': personal_name_list}

    try:
      #reponse returns a json structure/response object
      response = requests.request("POST", NAMSOR_URL, json=payload, headers=headers)

      if response.status_code == 200:
        print('Successful request')
      else:
        print('Request was unsuccessful: ', response.status_code)

    except requests.exceptions.RequestException as e:
      print('Error making the request: ', e)

    write_namsor_data_to_file(response)

    return response





def build_author_names_and_gender_dict(namsor_data):
    '''
    :params: namsor_data(reponse object) containing data given back by the namsor api
    :return: names_and_gender_dict - dictionary containing key of names, and value is a sub dictionary containing gender and probability

    we build this dicitonary in order to iterate over it and add the relevant values to the excel file
    '''

    #data = namsor_data.json()
    data = namsor_data
    names_and_gender_dict = {}

    for subdict in data['personalNames']:
        names_and_gender_dict[subdict['name']] = {'likelyGender':subdict['likelyGender'], 'probabilityCalibrated':subdict['probabilityCalibrated']}


    return names_and_gender_dict


def update_xlsx_with_gender_data(df, names_and_gender_dict):
    '''
    :param df: pandas data frame containing the information from the LawReview xlsx file
           names_and_gender_dict: dictionary containing the
    :return:
    update xlsx file with our gender data
    '''

    #creating new columns
    df['author_genders'] = ''
    df['probability_calibrated'] = ''
    index = 0

    print(f'max num authors: {max_num_authors}')
    #create columns for each author entry
    for i in range(max_num_authors):
      df[f'author{i+1}_gender'] = ''
      df[f'author{i+1}_probability'] = ''

    #create columns for total male total female and total authors
    df['total_male'] = ''
    df['total_female'] = ''
    df['total_authors'] = ''

    #itertuples() has better performance than iterrows()
    for row in df.itertuples(index=False):
        gender_list = []
        probability_list = []
        male_count = 0
        female_count = 0
        total_author_count = 0
        current_author = str(row.authors)
        #split for multiple authors

        if ';' in current_author:
          current_authors_list = current_author.split(';')
        else:
          current_authors_list = current_author.split(' AND ')

        #find corresponding gender for each author
        for name in current_authors_list:
          if name.strip() in names_and_gender_dict:
            #create an ordered list of genders
            gender_list.append(names_and_gender_dict[name.strip()]['likelyGender'])
            probability_list.append(str(names_and_gender_dict[name.strip()]['probabilityCalibrated']))

        #round the probabilities to two decimal points
        for i in range(len(probability_list)):
          probability_list[i] = str(round(float(probability_list[i]), 4))

        #find number of male and female authors
        for i in range(len(gender_list)):
          if gender_list[i] == 'male':
            male_count += 1
          else:
            female_count += 1


        #add the genders in order dilimited by a ';'
        #print(gender_list)
        df.at[index, 'author_genders'] = '; '.join(gender_list)
        df.at[index, 'probability_calibrated'] = '; '.join(probability_list)


        #Add the authors gender and probability to corresponding columns
        for i in range(len(gender_list)):
          df.at[index, f'author{i+1}_gender'] = gender_list[i]
          df.at[index, f'author{i+1}_probability'] = probability_list[i]

        #add the number of male and female and total
        df.at[index, 'total_male'] = male_count
        df.at[index, 'total_female'] = female_count
        df.at[index,'total_authors'] = male_count + female_count

        index += 1

    #write to a copy
    with pd.ExcelWriter(f'{xlsx_file}_Copy.xlsx', mode='w', engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='Sheet1')





In [8]:
def create_directory_if_not_exists(directory):
  '''
  creates a new directory if it doesnt already exist
  '''

  if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Created Directory: {directory}')
  else:
    print(f'Directory already exists: {directory}')

def write_namsor_data_to_file(response):
  '''
  writes namsor json data into a file for saving
  this way the API only has to be called once
  '''
  create_directory_if_not_exists(namsor_dir)
  with open(f'{namsor_dir}/namsor_data.json', 'w') as file:
    file.write(response.text)

def read_namsor_data_from_file():
  '''
  reads the json data from the existing file
  '''
  try:
    with open(f'{namsor_dir}/namsor_data.json', 'r') as file:
      response = json.load(file)
  except FileNotFoundError:
    print('File not found')

  return response

def find_max_num_authors(author_list):
  global max_num_authors
  if(len(author_list) > max_num_authors):
    max_num_authors = len(author_list)






# MAIN

In [9]:
df = pd.read_excel(xlsx_file)
author_names_dict = get_author_names_from_data(df)
print(len(author_names_dict))

print (author_names_dict)

skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: DAVID S. LAW ; WEN-CHEN CHANG
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: NORMAN B. PAGE
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ROGER D. BLAIR ; WILLIAM H. PAGE
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: ANON
skipping name: AN

In [10]:
print(author_names_dict)

{'RALPH W. JOHNSON': None, 'RENNARD STRICKLAND': None, 'CHARLES WILKINSON': None, 'JOHN F. COVERDALE': None, 'DANIEL H. FOOTE': None, 'RANDALL S. THOMAS': None, 'KENNETH J. MARTIN': None, 'MAUREEN M. CYR': None, 'SHELLEY M. HALL': None, 'KATRINA R. KELLY': None, 'MATTHEW J. MCCLOSKEY': None, 'JULIE S. NORTHUP': None, 'EDGAR SARGENT': None, 'MICHAEL I. SWYGERT': None, 'KATHERINE E. YANES': None, 'RUSSELL B. KOROBKIN': None, 'THOMAS S. ULEN': None, 'RICHARD O. ZERBE JR.': None, 'SYLVIA A. LAW': None, 'ELIZABETH C. CHAMBERS': None, 'CASEY J. LITTLE': None, 'ANDREA LOVELL': None, 'DAVID J. ROBERTS': None, 'MICHAEL A. GEIST': None, 'ERICA W. HARRIS': None, 'ANTHONY LEWIS': None, "ROBERT M. O'NEIL": None, 'DENISE A. BANASZEWSKI': None, 'KRISTIANA L. FARRIS': None, 'MICHELLE D. GAINES': None, 'ADRIENNE T. MCCOY': None, 'TRACI S. STRATTON': None, 'THOMAS A. PIRAINO JR.': None, 'JENNIFER E. SPRENG': None, 'AARON H. CAPLAN': None, 'LEV S. BATALOV': None, 'LISA D. GOULD': None, 'KATHLEEN HAGGARD'

In [11]:
#if we are running this for the first time on a new law review, call the namsor_request. Otherwise, read the data from the file
response = namsor_request(author_names_dict)
response = read_namsor_data_from_file()


Successful request
Created Directory: /content/drive/MyDrive/LawRevProject/Data/Washington Law Rev/Namsor


In [12]:
print(response)

{'personalNames': [{'script': 'LATIN', 'id': '9b8af4b4-a502-4423-bdb2-21e0a35e0fe8', 'name': 'RALPH W. JOHNSON', 'likelyGender': 'male', 'genderScale': -0.649327776581027, 'score': 6.96624211202554, 'probabilityCalibrated': 0.8246638882905135}, {'script': 'LATIN', 'id': '83222626-b324-4429-a421-ca40b3fd3da5', 'name': 'RENNARD STRICKLAND', 'likelyGender': 'male', 'genderScale': -0.8346146908205252, 'score': 8.580699445817856, 'probabilityCalibrated': 0.9173073454102626}, {'script': 'LATIN', 'id': 'b2f407db-499a-4c3a-abe7-17f75f340bd4', 'name': 'CHARLES WILKINSON', 'likelyGender': 'male', 'genderScale': -0.8881526084278368, 'score': 9.375958840397452, 'probabilityCalibrated': 0.9440763042139184}, {'script': 'LATIN', 'id': '17784318-50be-40b4-939c-ca156fc976a1', 'name': 'JOHN F. COVERDALE', 'likelyGender': 'male', 'genderScale': -0.9259686013332966, 'score': 11.115045487273129, 'probabilityCalibrated': 0.9629843006666483}, {'script': 'LATIN', 'id': '3f1e2752-1978-4f94-95e3-904b80087b36', 

In [13]:
names_and_gender_dict = build_author_names_and_gender_dict(response)


In [14]:
print(names_and_gender_dict)

{'RALPH W. JOHNSON': {'likelyGender': 'male', 'probabilityCalibrated': 0.8246638882905135}, 'RENNARD STRICKLAND': {'likelyGender': 'male', 'probabilityCalibrated': 0.9173073454102626}, 'CHARLES WILKINSON': {'likelyGender': 'male', 'probabilityCalibrated': 0.9440763042139184}, 'JOHN F. COVERDALE': {'likelyGender': 'male', 'probabilityCalibrated': 0.9629843006666483}, 'DANIEL H. FOOTE': {'likelyGender': 'male', 'probabilityCalibrated': 0.8706316757227106}, 'RANDALL S. THOMAS': {'likelyGender': 'male', 'probabilityCalibrated': 0.8719423031895612}, 'KENNETH J. MARTIN': {'likelyGender': 'male', 'probabilityCalibrated': 0.7813648079809353}, 'MAUREEN M. CYR': {'likelyGender': 'female', 'probabilityCalibrated': 0.6851231769854934}, 'SHELLEY M. HALL': {'likelyGender': 'female', 'probabilityCalibrated': 0.6898972469082536}, 'KATRINA R. KELLY': {'likelyGender': 'female', 'probabilityCalibrated': 0.8984861870254648}, 'MATTHEW J. MCCLOSKEY': {'likelyGender': 'male', 'probabilityCalibrated': 0.95410

In [15]:
update_xlsx_with_gender_data(df, names_and_gender_dict)

max num authors: 7


---- Think about how you can vectorize some of the operations with the dataframes.

In [16]:
'''

def get_author_names_from_data(df):

    :param df: dataframe containing the original data from law review xlsx file
    :return: dictionary containing just the author names extracted from the file to give to the Namsor API

    names = []
    namesDict = {}
    skip_names_list = invalid_names[unique_key]
    index = 0

    for row in df.itertuples():
        name = str(row.authors)
        name = name.strip()

        for skip_name in skip_names_list:
          if skip_name.lower() in name.lower() or name.lower() == 'nan' or name == None:
            index += 1
            continue
        #some entries are incorrect, so we skip them
        # if name in skip_names_list or name.lower() == 'nan' or name == None:
        #     index += 1
        #     continue

        #split multiple authors and enter each author into dictionary
        #split on ';' first... sometimes the names are separated by 'AND', so check if it is separated by 'AND'
        if ';' in name:
          current_authors_list = name.split(';')
        else:
          current_authors_list = name.split(' AND ')


        find_max_num_authors(current_authors_list)

        for current_author in current_authors_list:
          namesDict[current_author.strip()] = None
          names.append(current_author)



        index += 1


    return namesDict
'''

"\n\ndef get_author_names_from_data(df):\n\n    :param df: dataframe containing the original data from law review xlsx file\n    :return: dictionary containing just the author names extracted from the file to give to the Namsor API\n\n    names = []\n    namesDict = {}\n    skip_names_list = invalid_names[unique_key]\n    index = 0\n\n    for row in df.itertuples():\n        name = str(row.authors)\n        name = name.strip()\n\n        for skip_name in skip_names_list:\n          if skip_name.lower() in name.lower() or name.lower() == 'nan' or name == None:\n            index += 1\n            continue\n        #some entries are incorrect, so we skip them\n        # if name in skip_names_list or name.lower() == 'nan' or name == None:\n        #     index += 1\n        #     continue\n\n        #split multiple authors and enter each author into dictionary\n        #split on ';' first... sometimes the names are separated by 'AND', so check if it is separated by 'AND'\n        if ';' in